## Scenario 3: Multiple data scientists working on multiple ML models
MLflow setup:

- Tracking server: yes, remote server (EC2).
- Backend store: postgresql database.
- Artifacts store: s3 bucket.

  
The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file mlflow_on_aws.md to create a new AWS account and launch the tracking server.

In [9]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "tejash" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-35-174-154-213.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [10]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-35-174-154-213.compute-1.amazonaws.com:5000'


In [11]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-mlops-zoomcamp/1', creation_time=1720986789477, experiment_id='1', last_update_time=1720986789477, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-mlops-zoomcamp/0', creation_time=1720986616851, experiment_id='0', last_update_time=1720986616851, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 's3://mlflow-artifacts-remote-mlops-zoomcamp/1/ceff5e19fa05412cadb8b1be89462a49/artifacts'


In [13]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-mlops-zoomcamp/1', creation_time=1720986789477, experiment_id='1', last_update_time=1720986789477, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-mlops-zoomcamp/0', creation_time=1720986616851, experiment_id='0', last_update_time=1720986616851, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [14]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [15]:
client.search_registered_models()

[]

In [16]:
run_id = "ceff5e19fa05412cadb8b1be89462a49"
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/07/14 20:05:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1720987524168, current_stage='None', description='', last_updated_timestamp=1720987524168, name='iris-classifier', run_id='ceff5e19fa05412cadb8b1be89462a49', run_link='', source='s3://mlflow-artifacts-remote-mlops-zoomcamp/1/ceff5e19fa05412cadb8b1be89462a49/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>